In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn import preprocessing
import nltk
import csv
import re
import math
from pprint import pprint
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt


import networkx as nx
from networkx import betweenness_centrality
from networkx import edge_betweenness_centrality
from networkx import load_centrality
from networkx import eigenvector_centrality

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

import en_core_web_sm

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Test Split

In [3]:
train_set = pd.read_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/temp_before_graph_features_train_set_March_20.csv')
test_set = pd.read_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/temp_before_graph_features_test_set_March_20.csv')

In [4]:
train_set.head()

,Unnamed: 0,source_id,target_id,label,source_authors,target_authors,publication_year_diff,source_journal,target_journal,same_journal,...,target_hub_score,source_authority_score,target_authority_score,source_cluster_coeff,target_cluster_coeff,source_out_centrality,target_in_centrality,source_avg_neigh_degree,target_avg_neigh_degree,feature_n
0,0,9510123,9502114,1,unknown,"['w. kim', 'j. lee', 'y. park']",0.0,Phys.Lett.,Phys.Lett.,1,...,5.928285e-07,0.000018,0.000012,0.007905,0.008547,0.000324,0.000720,20.777778,18.285714,2.334275e-07
1,1,9707075,9604178,1,"['l.e.ibanez', 'a.m.uranga']","['a. dabholkar', 'j. park']",1.0,unknown,Nucl.Phys.,0,...,9.071772e-05,0.000050,0.000371,0.166022,0.125799,0.002701,0.004754,32.480000,26.025000,1.283851e-05
2,2,9312155,9506142,0,"['p. kleban', 'i. vassileva']",['v. moretti'],-2.0,Phys.Rev.Lett.,Class.Quant.Grav.,0,...,2.375408e-06,0.000003,0.000002,0.006536,0.008658,0.000288,0.000288,26.125000,21.214286,8.299645e-08
3,3,9911255,302165,0,"['p. argyres', 's. pell,']",['t. hollowood'],-4.0,JHEP,unknown,0,...,1.170214e-05,0.000030,0.000017,0.138138,0.081439,0.000972,0.000432,21.851852,29.000000,4.201695e-07
4,4,9701033,209076,0,unknown,"['e. gravanis', 's. willison']",-5.0,Phys.Lett.,unknown,0,...,1.728026e-05,0.000012,0.000014,0.016043,0.104533,0.000648,0.000432,28.444444,28.400000,2.801130e-07


In [5]:
# 去掉unnamed
train_set2 = train_set.iloc[:, 1:]
test_set2 = test_set.iloc[:, 1:]

In [6]:
# separate features from labels:
X = train_set.loc[:, (train_set.columns != 'label') & 
                     (train_set.columns != 'common_authors') & 
                     (train_set.columns != 'source_authors') & 
                     (train_set.columns != 'target_authors') & 
                     (train_set.columns != 'source_journal') & 
                     (train_set.columns != 'target_journal')]
y = train_set['label']
# y.astype(int)

In [ ]:
test = test_set2.loc[:, (test_set2.columns != 'label') & 
                        (test_set2.columns != 'common_authors') & 
                        (test_set2.columns != 'source_authors') & 
                        (test_set2.columns != 'target_authors') & 
                        (test_set2.columns != 'source_journal') & 
                        (test_set2.columns != 'target_journal')]
test.head()

In [ ]:
# # final feature correlation
# ff = X.copy()
# ff['label'] = y
# plt.figure(figsize=(14,12))
# sns.heatmap(X.corr(),
#             vmax=0.5,
#             square=True,
#             annot=True)

In [ ]:
train_set2.head()

In [8]:
test_set2['label']

0       -1.0
1       -1.0
2       -1.0
3       -1.0
4       -1.0
        ... 
32643   -1.0
32644   -1.0
32645   -1.0
32646   -1.0
32647   -1.0
Name: label, Length: 32648, dtype: float64

In [9]:
train_set2['label']

0         1
1         1
2         0
3         0
4         0
         ..
615507    1
615508    0
615509    0
615510    1
615511    1
Name: label, Length: 615512, dtype: int64

##  XGB

In [ ]:
test_set2.shape

(32648, 89)

In [ ]:
test = test_set2.loc[:, (test_set2.columns != 'label') & 
                        (test_set2.columns != 'common_authors') & 
                        (test_set2.columns != 'source_authors') & 
                        (test_set2.columns != 'target_authors') & 
                        (test_set2.columns != 'source_journal') & 
                        (test_set2.columns != 'target_journal')]
test.head()

,source_id,target_id,publication_year_diff,same_journal,similarity_title,similarity_abstract,similarity_author,0_source,1_source,2_source,...,target_hub_score,source_authority_score,target_authority_score,source_cluster_coeff,target_cluster_coeff,source_out_centrality,target_in_centrality,source_avg_neigh_degree,target_avg_neigh_degree,feature_n
0,9807076,9807139,0.0,0,0.066711,0.089544,0.0,-0.276091,-0.548179,-0.611863,...,0.000036,0.000146,0.000005,0.087579,0.098151,0.000648,0.000432,23.277778,27.615385,2.801130e-07
1,109162,1182,1.0,0,0.308697,0.188503,0.0,-0.063945,-0.019596,0.012476,...,0.000021,0.000088,0.000048,0.067997,0.193548,0.007778,0.001729,35.430556,28.071429,1.344542e-05
2,9702187,9510135,2.0,1,0.191229,0.140174,0.0,0.852248,0.056141,-0.029177,...,0.000102,0.000738,0.002486,0.058951,0.025157,0.000936,0.026540,24.461538,21.300000,2.484965e-05
3,111048,110115,0.0,1,0.042662,0.133604,0.0,-0.246824,-0.279106,0.775912,...,0.000073,0.000021,0.000043,0.173677,0.141997,0.001729,0.001008,34.416667,38.392857,1.742925e-06
4,9910176,9410073,5.0,0,0.169901,0.351626,0.0,-0.068009,-0.021185,0.013551,...,0.000011,0.000022,0.000503,0.016667,0.043384,0.000504,0.005582,19.571429,21.882353,2.814098e-06


In [ ]:
# making sure the test and the train files have same sequence of columns

# test = test[X.columns]

from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate

# defining the base model
xgb_model_base = XGBClassifier(n_estimators = 100)

# printing the cross validation scores for the classifier
scores = cross_validate(xgb_model_base, X, y.values.ravel(), 
                        scoring='f1', 
                        cv=3,
                        n_jobs = -1,
                        verbose=True) # n_jobs is the number of cpus to use -1 => all

# fitting on the training data
xgb_model_base.fit(X, y.values.ravel())


# predicting the outcome from the final 
predictions = xgb_model_base.predict(test)

# write out
out_df = test_set.copy()
data = {'id': list(out_df.index), 'category': predictions}
final_df = pd.DataFrame(data)


# 3: write file out
final_df.to_csv('submission.csv',index=False, sep=',')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
# defining the search grid
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from xgboost.sklearn import XGBClassifier

xgb_regressor = xgb.XGBRegressor()

random_grid = {
     "n_estimators"     : [int(x) for x in np.linspace(50, 600, num = 20)],
     "learning_rate"    : [0.01, 0.02, 0.05, 0.10 ] ,
     "max_depth"        : [ 6, 8, 10, 12, 15, 20],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.3, 0.4, 0.7, 0.9 ],
     "colsample_bytree" : [ 0.05, 0.1, 0.3, 0.4]}

# Use the random grid to search for best hyperparameters

# First create the base model to tune
xgb_model = XGBClassifier()

# Random search of parameters
xgb_random = RandomizedSearchCV(estimator = xgb_model, param_distributions = random_grid,
n_iter = 10, cv = 3, verbose=2, random_state=42 ,n_jobs = -1, scoring = 'f1_weighted')

optimised_xgb_random = xgb_random.best_estimator_



# printing the cross validation scores for the classifier
scores = cross_validate(optimised_xgb_random, X, y.values.ravel(), scoring='f1', 
                        cv=3,n_jobs = -1 ) # n_jobs is the number of cpus to use -1 => all


# fitting on the training data
xgb_model.fit(X, y.values.ravel())

# predicting the outcome from the final 
optimised_xgb_random.predict(test)

# write out
out_df = test_set.copy()
data = {'id': list(out_df.index), 'category': predictions}
final_df = pd.DataFrame(data)


# 3: write file out
final_df.to_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/submission_XGB_March_20_1.csv', index=False, sep=',')

## SVM

In [ ]:
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from xgboost.sklearn import XGBClassifier

# SVM has a zero tolerance towards null values, hence replacing them by 0

XVM = X.fillna(value=0)
test_SVM = test.fillna(value=0)

clf = LinearSVC(tol=1e-4)


# printing the cross validation scores for the classifier
scores = cross_validate(clf, XVM, y, 
                        scoring='f1', 
                        cv=10,
                        n_jobs = -1, 
                        verbose=True) # n_jobs is the number of cpus to use -1 => all

# fitting on the training data
clf.fit(XVM, y)

In [18]:
test_set = pd.read_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/temp_before_graph_features_test_set_March_20.csv')

test2 = test_set.loc[:, (test_set.columns != 'label') & 
                        (test_set.columns != 'common_authors') & 
                        (test_set.columns != 'source_authors') & 
                        (test_set.columns != 'target_authors') & 
                        (test_set.columns != 'source_journal') & 
                        (test_set.columns != 'target_journal')]

test_SVM = test2.fillna(value=0)

In [20]:
# predicting the outcome from the final 
prediction_clf = clf.predict(test_SVM)

# write out
out_df = test_set.copy()
data = {'id': list(out_df.index), 'category': prediction_clf}
final_df = pd.DataFrame(data)


# 3: write file out
final_df.to_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/submission_SVM_March_20_1.csv',index=False, sep=',')

In [23]:
final_df.category.value_counts()

1    32613
0       35
Name: category, dtype: int64

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# 1: retrain the complete model -> don't forget to change this to optimal one @ end
final_model = RandomForestClassifier()
final_model.fit(X, y)

# 2: predict on the test set
final_test_set = test_set.loc[:,(test_set.columns != 'source_authors') & 
                                (test_set.columns != 'common_authors') & 
                                (test_set.columns != 'target_authors') & 
                                (test_set.columns != 'label') & 
                                (test_set.columns != 'source_journal') & 
                                (test_set.columns != 'target_journal')]
predictions = final_model.predict(final_test_set)

# write out
out_df = test_set.copy()
data = {'id': list(out_df.index), 'category': predictions}
final_df = pd.DataFrame(data)


# 3: write file out
final_df.to_csv('/content/drive/MyDrive/DSBA M2/2 MLNS/Kaggle/my own work/submission_RF_March_20_1.csv',index=False, sep=',')

# plot the feature importance
feat_importances = pd.Series(final_model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()